# **Install Dependencies**

In [ ]:
# 📦 Install dependencies
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'
!pip install -q pycocotools

# 🔧 Setup: Import
import os
from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
import random
import cv2
from matplotlib import pyplot as plt

# **Register Dataset**

In [ ]:
# Set root folder di Google Drive
dataset_root = "/content/drive/MyDrive/210411100014_Taufiqu Reza Yoga Pratama/Dataset/dataset_new"

# Buat path lengkap ke JSON dan folder gambar
train_json = os.path.join(dataset_root, "train/_annotations.coco.json")
val_json = os.path.join(dataset_root, "valid/_annotations.coco.json")
test_json = os.path.join(dataset_root, "test/_annotations.coco.json")
train_img_dir = os.path.join(dataset_root, "train")
val_img_dir = os.path.join(dataset_root, "valid")
test_img_dir = os.path.join(dataset_root, "test")

# ✅ Cek apakah file dan folder ada
print("Train JSON exists:", os.path.exists(train_json))
print("Train Image Dir exists:", os.path.exists(train_img_dir))

# 🗂 Daftarkan dataset ke Detectron2
register_coco_instances("roboflow_train", {}, train_json, train_img_dir)
register_coco_instances("roboflow_val", {}, val_json, val_img_dir)
register_coco_instances("roboflow_test", {}, test_json, test_img_dir)

# Epoch 30_1e-3

In [ ]:
from detectron2 import model_zoo

# ⚙️ Konfigurasi RetinaNet
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/retinanet_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("roboflow_train",)
cfg.DATASETS.TEST = ("roboflow_val",)
cfg.DATALOADER.NUM_WORKERS = 4

cfg.MODEL.WEIGHTS = "detectron2://ImageNetPretrained/MSRA/R-50.pkl"
cfg.SOLVER.IMS_PER_BATCH = 16
cfg.SOLVER.BASE_LR = 0.001  # Untuk Adam, bisa 1e-3 – 1e-4
cfg.SOLVER.MAX_ITER = 1680  # Ubah sesuai jumlah data dan epoch yang diinginkan
cfg.SOLVER.STEPS = []  # Jangan gunakan step scheduler untuk Adam
cfg.SOLVER.WEIGHT_DECAY = 0.0001
cfg.SOLVER.WEIGHT_DECAY_BIAS = 0.0
cfg.SOLVER.WEIGHT_DECAY_NORM = 0.0  # Tambahan penting

cfg.TEST.EVAL_PERIOD = 500  # Evaluasi setiap 500 iterasi
cfg.MODEL.RETINANET.NUM_CLASSES = 4  # Jumlah kelas sebenarnya (tanpa background)

# 📂 Output directory
cfg.OUTPUT_DIR = "/content/drive/MyDrive/210411100014_Taufiqu Reza Yoga Pratama/Model/new/epoch_30_1e-3_Adam"
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

In [ ]:
from detectron2.engine.hooks import BestCheckpointer
from detectron2.evaluation import COCOEvaluator
from detectron2.data import build_detection_test_loader
from detectron2.engine import DefaultTrainer
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.utils.events import EventStorage

import torch

# 🧠 Custom Trainer class
class MyTrainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
        return COCOEvaluator(dataset_name, cfg, False, output_folder)

    @classmethod
    def build_optimizer(cls, cfg, model):
        """
        Override default SGD optimizer with Adam
        """
        params = []
        for name, param in model.named_parameters():
            if not param.requires_grad:
                continue
            lr = cfg.SOLVER.BASE_LR
            if "bias" in name:
                weight_decay = cfg.SOLVER.WEIGHT_DECAY_BIAS
            elif "norm" in name:
                weight_decay = cfg.SOLVER.WEIGHT_DECAY_NORM  # Tambahkan decay norm
            else:
                weight_decay = cfg.SOLVER.WEIGHT_DECAY
            params.append({"params": [param], "lr": lr, "weight_decay": weight_decay})

        optimizer = torch.optim.Adam(params, lr=cfg.SOLVER.BASE_LR)
        return optimizer

    def build_hooks(self):
        hooks = super().build_hooks()
        hooks.insert(
            -1,
            BestCheckpointer(
                cfg.TEST.EVAL_PERIOD,
                checkpointer=DetectionCheckpointer(self.model, cfg.OUTPUT_DIR),
                val_metric="bbox/AP",
                mode="max",
                file_prefix="best_model"
            )
        )
        return hooks

# Inisialisasi trainer dan mulai training
trainer = MyTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

# Epoch 30_1e-5

In [ ]:
from detectron2 import model_zoo

# ⚙️ Konfigurasi RetinaNet
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/retinanet_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("roboflow_train",)
cfg.DATASETS.TEST = ("roboflow_val",)
cfg.DATALOADER.NUM_WORKERS = 4

cfg.MODEL.WEIGHTS = "detectron2://ImageNetPretrained/MSRA/R-50.pkl"
cfg.SOLVER.IMS_PER_BATCH = 16
cfg.SOLVER.BASE_LR = 0.00001
cfg.SOLVER.MAX_ITER = 1680  # Ubah sesuai jumlah data dan epoch yang diinginkan
cfg.SOLVER.STEPS = []  # Jangan gunakan step scheduler untuk Adam
cfg.SOLVER.WEIGHT_DECAY = 0.0001
cfg.SOLVER.WEIGHT_DECAY_BIAS = 0.0
# cfg.SOLVER.WEIGHT_DECAY_NORM = 0.0  # Tambahan penting

cfg.TEST.EVAL_PERIOD = 500  # Evaluasi setiap 500 iterasi
cfg.MODEL.RETINANET.NUM_CLASSES = 4  # Jumlah kelas sebenarnya (tanpa background)

# 📂 Output directory
cfg.OUTPUT_DIR = "/content/drive/MyDrive/210411100014_Taufiqu Reza Yoga Pratama/Model/new/epoch_30_1e-5_Adam"
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

In [ ]:
from detectron2.engine.hooks import BestCheckpointer
from detectron2.evaluation import COCOEvaluator
from detectron2.data import build_detection_test_loader
from detectron2.engine import DefaultTrainer
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.utils.events import EventStorage

import torch

# 🧠 Custom Trainer class
class MyTrainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
        return COCOEvaluator(dataset_name, cfg, False, output_folder)

    @classmethod
    def build_optimizer(cls, cfg, model):
        """
        Override default SGD optimizer with Adam
        """
        params = []
        for name, param in model.named_parameters():
            if not param.requires_grad:
                continue
            lr = cfg.SOLVER.BASE_LR
            if "bias" in name:
                weight_decay = cfg.SOLVER.WEIGHT_DECAY_BIAS
            elif "norm" in name:
                weight_decay = cfg.SOLVER.WEIGHT_DECAY_NORM  # Tambahkan decay norm
            else:
                weight_decay = cfg.SOLVER.WEIGHT_DECAY
            params.append({"params": [param], "lr": lr, "weight_decay": weight_decay})

        optimizer = torch.optim.Adam(params, lr=cfg.SOLVER.BASE_LR)
        return optimizer

    def build_hooks(self):
        hooks = super().build_hooks()
        hooks.insert(
            -1,
            BestCheckpointer(
                cfg.TEST.EVAL_PERIOD,
                checkpointer=DetectionCheckpointer(self.model, cfg.OUTPUT_DIR),
                val_metric="bbox/AP",
                mode="max",
                file_prefix="best_model"
            )
        )
        return hooks

# Inisialisasi trainer dan mulai training
trainer = MyTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

**Save Konfigurasi**

In [ ]:
with open(os.path.join(cfg.OUTPUT_DIR, "config.yaml"), "w") as f:
    f.write(cfg.dump())

**Test Prediksi**

In [ ]:
# 🧠 Load the trained model
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "best_model.pth")  # Load the weights from your training
#cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # Set threshold to 50% for detection

from detectron2.engine import DefaultPredictor

predictor = DefaultPredictor(cfg)

# 📸 Read the image
#image_path = '/content/valid/00000732_005_jpg.rf.3541c4c659fec2d5d003b4fb7a380dea.jpg'
image_path = "/content/drive/MyDrive/210411100014_Taufiqu Reza Yoga Pratama/Dataset/test/9ed1f91369552618456da255b02820b3_jpg.rf.17cd671024249e0b1f3a9e36be41a928.jpg"
im = cv2.imread(image_path)

# 🧠 Run inference
outputs = predictor(im)

# Filter instances with confidence
confidence = 0.5
instances = outputs["instances"]
scores = instances.scores
filtered_instances = instances[scores > confidence]

# 🖼️ Visualize the results
v = Visualizer(im[:, :, ::-1], MetadataCatalog.get("roboflow_train"), scale=1)
v = v.draw_instance_predictions(filtered_instances.to("cpu"))

# Show the image with filtered instances
plt.figure(figsize=(10, 10))
plt.imshow(v.get_image())
plt.show()

# Optionally, print out the number of detections
print(f"Detected {len(filtered_instances)} objects with confidence > {confidence*100}%.")


**Confusion Matriks**

In [ ]:
def calculate_iou(box1, box2):
    """
    box: [xmin, ymin, xmax, ymax]
    """
    x1, y1, x2, y2 = box1
    x1g, y1g, x2g, y2g = box2

    xi1 = max(x1, x1g)
    yi1 = max(y1, y1g)
    xi2 = min(x2, x2g)
    yi2 = min(y2, y2g)
    inter_area = max(xi2 - xi1, 0) * max(yi2 - yi1, 0)

    box1_area = (x2 - x1) * (y2 - y1)
    box2_area = (x2g - x1g) * (y2g - y1g)
    union_area = box1_area + box2_area - inter_area

    if union_area == 0:
        return 0
    else:
        return inter_area / union_area


In [ ]:
import os
import cv2
import numpy as np
import json
import matplotlib.pyplot as plt
from tqdm import tqdm
from detectron2.engine import DefaultPredictor
from detectron2.data import MetadataCatalog
from detectron2.utils.visualizer import Visualizer
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# 🧠 Load model
# cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "best_model.pth")
cfg.MODEL.WEIGHTS = "/content/drive/MyDrive/210411100014_Taufiqu Reza Yoga Pratama/Model/new/epoch_30_1e-5_Adam/best_model.pth"
predictor = DefaultPredictor(cfg)

# 📚 Prepare dataset paths
test_image_dir = "/content/drive/MyDrive/210411100014_Taufiqu Reza Yoga Pratama/Dataset/dataset_new/test"
coco_annotation_path = "/content/drive/MyDrive/210411100014_Taufiqu Reza Yoga Pratama/Dataset/dataset_new/test/_annotations.coco.json"

# Load COCO ground truth
with open(coco_annotation_path) as f:
    coco_data = json.load(f)

# Build image_id to filename mapping
id_to_filename = {img['id']: img['file_name'] for img in coco_data['images']}

# Build annotations mapping
annotations = coco_data['annotations']
gt_dict = {}  # {filename: (boxes, labels)}

for ann in annotations:
    image_id = ann['image_id']
    filename = id_to_filename[image_id]
    bbox = ann['bbox']  # [x, y, width, height]
    category_id = ann['category_id']

    if filename not in gt_dict:
        gt_dict[filename] = {'boxes': [], 'labels': []}

    # Convert bbox format to [xmin, ymin, xmax, ymax]
    xmin, ymin, w, h = bbox
    xmax = xmin + w
    ymax = ymin + h

    gt_dict[filename]['boxes'].append([xmin, ymin, xmax, ymax])
    gt_dict[filename]['labels'].append(category_id)

# Build category_id to class_name mapping
category_id_to_name = {cat['id']: cat['name'] for cat in coco_data['categories']}
class_names = [category_id_to_name[i] for i in sorted(category_id_to_name.keys())]

# 🔥 Start evaluating
y_true = []
y_pred = []
iou_threshold = 0.5
confidence_threshold = 0.5

test_images = sorted(os.listdir(test_image_dir))
test_images = [img for img in test_images if img.endswith('.jpg') or img.endswith('.png')]

for img_file in tqdm(test_images, desc="Processing Test Images"):
    # Read image
    img_path = os.path.join(test_image_dir, img_file)
    img = cv2.imread(img_path)

    if img_file not in gt_dict:
        continue  # skip images without ground truth

    gt_boxes = np.array(gt_dict[img_file]['boxes'])
    gt_classes = np.array(gt_dict[img_file]['labels'])

    # Predict
    outputs = predictor(img)
    instances = outputs["instances"].to("cpu")

    pred_boxes = instances.pred_boxes.tensor.numpy()
    pred_classes = instances.pred_classes.numpy()
    pred_scores = instances.scores.numpy()

    # Filter predictions by confidence
    keep = pred_scores > confidence_threshold
    pred_boxes = pred_boxes[keep]
    pred_classes = pred_classes[keep]

    matched_gt = set()

    # Matching predicted boxes to ground truth boxes
    for pred_box, pred_class in zip(pred_boxes, pred_classes):
        best_iou = 0
        best_idx = -1
        for idx, gt_box in enumerate(gt_boxes):
            iou = calculate_iou(pred_box, gt_box)
            if iou > best_iou and idx not in matched_gt:
                best_iou = iou
                best_idx = idx
        if best_iou >= iou_threshold and best_idx != -1:
            y_true.append(gt_classes[best_idx])
            y_pred.append(pred_class)
            matched_gt.add(best_idx)
        else:
            # False positive
            y_true.append(-1)  # background
            y_pred.append(pred_class)

    # False negatives
    for idx, gt_class in enumerate(gt_classes):
        if idx not in matched_gt:
            y_true.append(gt_class)
            y_pred.append(-1)

# 🔥 Build Confusion Matrix
all_labels = sorted(list(set([lab for lab in y_true if lab != -1] + [lab for lab in y_pred if lab != -1])))
cm = confusion_matrix(y_true, y_pred, labels=all_labels)

class_labels = [category_id_to_name[i] for i in all_labels]

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_labels)
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix - RetinaNet Detectron2 (COCO Format)")
plt.show()


**Evaluasi Data Train**

In [ ]:
from detectron2.config import get_cfg
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.modeling import build_model

# Load config dan model
cfg = get_cfg()
cfg.merge_from_file("/content/drive/MyDrive/210411100014_Taufiqu Reza Yoga Pratama/Model/new/epoch_30_1e-5_Adam/config.yaml")
cfg.MODEL.WEIGHTS = "/content/drive/MyDrive/210411100014_Taufiqu Reza Yoga Pratama/Model/new/epoch_30_1e-5_Adam/best_model.pth"
cfg.MODEL.DEVICE = 'cpu'  # atau 'cpu'
cfg.DATASETS.TEST = ("roboflow_train",)
cfg.MODEL.RETINANET.SCORE_THRESH_TEST = 0.5

evaluator = COCOEvaluator("roboflow_train", cfg, False, output_dir="/content/drive/MyDrive/210411100014_Taufiqu Reza Yoga Pratama/Model/new/epoch_30_1e-5_Adam/eval_train")
val_loader = build_detection_test_loader(cfg, "roboflow_train")

model = build_model(cfg)
DetectionCheckpointer(model).load(cfg.MODEL.WEIGHTS)

results = inference_on_dataset(model, val_loader, evaluator)
print(results)

**Evaluasi Data Validasi**

In [ ]:
from detectron2.config import get_cfg
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.modeling import build_model

# Load config dan model
cfg = get_cfg()
cfg.merge_from_file("/content/drive/MyDrive/210411100014_Taufiqu Reza Yoga Pratama/Model/new/epoch_30_1e-5_Adam/config.yaml")
cfg.MODEL.WEIGHTS = "/content/drive/MyDrive/210411100014_Taufiqu Reza Yoga Pratama/Model/new/epoch_30_1e-5_Adam/best_model.pth"
cfg.MODEL.DEVICE = 'cpu'  # atau 'cpu'
cfg.DATASETS.TEST = ("roboflow_val",)
cfg.MODEL.RETINANET.SCORE_THRESH_TEST = 0.5

evaluator = COCOEvaluator("roboflow_val", cfg, False, output_dir="/content/drive/MyDrive/210411100014_Taufiqu Reza Yoga Pratama/Model/new/epoch_30_1e-5_Adam/eval_valid")
val_loader = build_detection_test_loader(cfg, "roboflow_val")

model = build_model(cfg)
DetectionCheckpointer(model).load(cfg.MODEL.WEIGHTS)

results = inference_on_dataset(model, val_loader, evaluator)
print(results)

**Evaluasi Data Testing**

In [ ]:
from detectron2.config import get_cfg
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.modeling import build_model

# Load config dan model
cfg = get_cfg()
cfg.merge_from_file("/content/drive/MyDrive/210411100014_Taufiqu Reza Yoga Pratama/Model/new/epoch_30_1e-5_Adam/config.yaml")
cfg.MODEL.WEIGHTS = "/content/drive/MyDrive/210411100014_Taufiqu Reza Yoga Pratama/Model/new/epoch_30_1e-5_Adam/best_model.pth"
cfg.MODEL.DEVICE = 'cpu'  # atau 'cpu'
cfg.DATASETS.TEST = ("roboflow_test",)
cfg.MODEL.RETINANET.SCORE_THRESH_TEST = 0.5

evaluator = COCOEvaluator("roboflow_test", cfg, False, output_dir="/content/drive/MyDrive/210411100014_Taufiqu Reza Yoga Pratama/Model/new/epoch_30_1e-5_Adam/eval_test")
val_loader = build_detection_test_loader(cfg, "roboflow_test")

model = build_model(cfg)
DetectionCheckpointer(model).load(cfg.MODEL.WEIGHTS)

results = inference_on_dataset(model, val_loader, evaluator)
print(results)

**Evaluasi Precision Recall F1-Score**

In [ ]:
from detectron2.evaluation.evaluator import DatasetEvaluator
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd

class PRF1Evaluator(DatasetEvaluator):
    def __init__(self, dataset_name, metadata, class_names=None):
        from detectron2.data import MetadataCatalog, DatasetCatalog
        self.metadata = MetadataCatalog.get(dataset_name)
        self.dataset_dicts = DatasetCatalog.get(dataset_name)
        self.class_names = class_names if class_names else self.metadata.thing_classes
        self.gts = []
        self.preds = []

        # Buat mapping image_id ke daftar ground truth class ID
        self.imgid_to_gt = {}
        for data in self.dataset_dicts:
            gt_classes = [ann["category_id"] for ann in data["annotations"]]
            self.imgid_to_gt[data["image_id"]] = gt_classes

    def reset(self):
        self.gts = []
        self.preds = []

    def process(self, inputs, outputs):
        for input, output in zip(inputs, outputs):
            img_id = input["image_id"]
            gt_classes = self.imgid_to_gt.get(img_id, [])
            pred_classes = output["instances"].pred_classes.cpu().numpy() if len(output["instances"]) > 0 else []

            # Samakan panjang prediksi dan ground truth
            if len(pred_classes) < len(gt_classes):
                pred_classes = list(pred_classes) + [-1] * (len(gt_classes) - len(pred_classes))
            elif len(pred_classes) > len(gt_classes):
                pred_classes = pred_classes[:len(gt_classes)]

            self.gts.extend(gt_classes)
            self.preds.extend(pred_classes)

    def evaluate(self):
        print("\n==== Precision, Recall, and F1-Score per Class ====\n")

        # Ambil hasil sebagai dictionary
        report_dict = classification_report(
            self.gts,
            self.preds,
            target_names=self.class_names,
            output_dict=True,
            digits=3,
            zero_division=0
        )

        # Konversi ke DataFrame
        df = pd.DataFrame(report_dict).transpose()
        df.index.name = 'Class'

        # Ambil dan simpan nilai akurasi, hapus dari tabel supaya tampil rapi
        accuracy = report_dict.get("accuracy", None)
        if "accuracy" in df.index:
            df.drop("accuracy", inplace=True)

        # Ubah support menjadi integer
        if 'support' in df.columns:
            df['support'] = df['support'].astype(int)

        # Urutkan kolom
        df = df[['precision', 'recall', 'f1-score', 'support']]

        # Cetak tabel
        print(df.to_string(float_format="%.3f"))

        # Cetak akurasi secara terpisah
        if accuracy is not None:
            print(f"\nOverall Accuracy: {accuracy:.3f}")

        # Return hanya data, tanpa tampilkan dict sebagai output Python
        return


**Data Train**

In [ ]:
class_names = ["Cardiomegaly", "Nodule/Mass", "Pneumothorax"]

evaluator_test = PRF1Evaluator("roboflow_train", cfg, class_names=class_names)
test_loader = build_detection_test_loader(cfg, "roboflow_train")
inference_on_dataset(trainer.model, test_loader, evaluator_test)


**Data Validasi**

In [ ]:
class_names = ["Cardiomegaly", "Nodule/Mass", "Pneumothorax"]

evaluator_test = PRF1Evaluator("roboflow_val", cfg, class_names=class_names)
test_loader = build_detection_test_loader(cfg, "roboflow_val")
inference_on_dataset(trainer.model, test_loader, evaluator_test)


**Data Testing**

In [ ]:
class_names = ["Cardiomegaly", "Nodule/Mass", "Pneumothorax"]

evaluator_test = PRF1Evaluator("roboflow_test", cfg, class_names=class_names)
test_loader = build_detection_test_loader(cfg, "roboflow_test")
inference_on_dataset(trainer.model, test_loader, evaluator_test)


# **Konfigurasi Train**

In [ ]:
import detectron2
print(detectron2.__version__)


In [ ]:
from detectron2 import model_zoo

# ⚙️ Konfigurasi RetinaNet
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/retinanet_R_50_FPN_3x.yaml"))

cfg.DATASETS.TRAIN = ("roboflow_train",)
cfg.DATASETS.TEST = ("roboflow_val",)
cfg.DATALOADER.NUM_WORKERS = 4

cfg.MODEL.WEIGHTS = "detectron2://ImageNetPretrained/MSRA/R-50.pkl"
cfg.MODEL.RETINANET.NUM_CLASSES = 4  # Jumlah kelas sebenarnya

# 🔧 Konfigurasi Optimizer Adam
cfg.SOLVER.BASE_LR = 5e-5  # Stabil dan umum untuk Adam
cfg.SOLVER.IMS_PER_BATCH = 16
cfg.SOLVER.MAX_ITER = 1680  # Jumlah iterasi total
cfg.SOLVER.STEPS = []  # Tidak pakai step decay
# cfg.SOLVER.WARMUP_ITERS = 0  # Tanpa warmup
cfg.SOLVER.WEIGHT_DECAY = 0.0001
cfg.SOLVER.WEIGHT_DECAY_BIAS = 0.0
# cfg.SOLVER.WEIGHT_DECAY_NORM = 0.0

# 📈 Evaluasi setiap 500 iterasi
cfg.TEST.EVAL_PERIOD = 500

# 📂 Direktori output
cfg.OUTPUT_DIR = "/content/drive/MyDrive/210411100014_Taufiqu Reza Yoga Pratama/Model/new/epoch_30_5e-5_Adam"
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

# **Training Skenario 1**

In [ ]:
from detectron2.engine.hooks import BestCheckpointer
from detectron2.evaluation import COCOEvaluator
from detectron2.data import build_detection_test_loader
from detectron2.engine import DefaultTrainer
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.utils.events import EventStorage

import torch

# 🧠 Custom Trainer class
class MyTrainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
        return COCOEvaluator(dataset_name, cfg, False, output_folder)

    @classmethod
    def build_optimizer(cls, cfg, model):
        """
        Override default SGD optimizer with Adam
        """
        params = []
        for name, param in model.named_parameters():
            if not param.requires_grad:
                continue
            lr = cfg.SOLVER.BASE_LR
            if "bias" in name:
                weight_decay = cfg.SOLVER.WEIGHT_DECAY_BIAS
            elif "norm" in name:
                weight_decay = cfg.SOLVER.WEIGHT_DECAY_NORM  # Tambahkan decay norm
            else:
                weight_decay = cfg.SOLVER.WEIGHT_DECAY
            params.append({"params": [param], "lr": lr, "weight_decay": weight_decay})

        optimizer = torch.optim.Adam(params, lr=cfg.SOLVER.BASE_LR)
        return optimizer

    def build_hooks(self):
        hooks = super().build_hooks()
        hooks.insert(
            -1,
            BestCheckpointer(
                cfg.TEST.EVAL_PERIOD,
                checkpointer=DetectionCheckpointer(self.model, cfg.OUTPUT_DIR),
                val_metric="bbox/AP",
                mode="max",
                file_prefix="best_model"
            )
        )
        return hooks

# Inisialisasi trainer dan mulai training
trainer = MyTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

# **Simpan Konfigurasi Model RetinaNet**

In [ ]:
with open(os.path.join(cfg.OUTPUT_DIR, "config.yaml"), "w") as f:
    f.write(cfg.dump())

# **Prediksi**

In [ ]:
# 🧠 Load the trained model
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "best_model.pth")  # Load the weights from your training
#cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # Set threshold to 50% for detection

from detectron2.engine import DefaultPredictor

predictor = DefaultPredictor(cfg)

# 📸 Read the image
#image_path = '/content/valid/00000732_005_jpg.rf.3541c4c659fec2d5d003b4fb7a380dea.jpg'
image_path = "/content/drive/MyDrive/210411100014_Taufiqu Reza Yoga Pratama/Dataset/test/9ed1f91369552618456da255b02820b3_jpg.rf.17cd671024249e0b1f3a9e36be41a928.jpg"
im = cv2.imread(image_path)

# 🧠 Run inference
outputs = predictor(im)

# Filter instances with confidence
confidence = 0.5
instances = outputs["instances"]
scores = instances.scores
filtered_instances = instances[scores > confidence]

# 🖼️ Visualize the results
v = Visualizer(im[:, :, ::-1], MetadataCatalog.get("roboflow_train"), scale=1)
v = v.draw_instance_predictions(filtered_instances.to("cpu"))

# Show the image with filtered instances
plt.figure(figsize=(10, 10))
plt.imshow(v.get_image())
plt.show()

# Optionally, print out the number of detections
print(f"Detected {len(filtered_instances)} objects with confidence > {confidence*100}%.")


# **Grafik Evaluasi Record Training**

In [ ]:
import json
import matplotlib.pyplot as plt

# Tentukan path ke file metrics.json
metrics_file = '/content/drive/MyDrive/210411100014_Taufiqu Reza Yoga Pratama/Model/new/epoch_30_1e-5_Adam/metrics.json'  # Ganti dengan path file Anda

# Menyaring data untuk total_loss
iterations = []
total_loss = []

# Membaca file JSON baris per baris
with open(metrics_file, 'r') as f:
    for line in f:
        # Memuat setiap baris JSON
        entry = json.loads(line)

        # Menyaring data untuk iteration dan total_loss
        if 'iteration' in entry and 'total_loss' in entry:
            iterations.append(entry['iteration'])
            total_loss.append(entry['total_loss'])

# Membuat grafik untuk total loss
plt.figure(figsize=(10, 6))
plt.plot(iterations, total_loss, label="Total Loss", color='tab:blue')

# Menambahkan judul dan label sumbu
plt.title('Total Loss over Iterations')
plt.xlabel('Iteration')
plt.ylabel('Total Loss')
plt.grid(True)

# Menampilkan grafik
plt.legend()
plt.show()

In [ ]:
import json
import matplotlib.pyplot as plt

# Path ke file metrics.json
metrics_file = '/content/drive/MyDrive/210411100014_Taufiqu Reza Yoga Pratama/Model/new/epoch_30_1e-5_Adam/metrics.json'  # Ganti sesuai path Anda

# Inisialisasi list untuk menyimpan data
iterations = []
total_loss = []
loss_cls = []
loss_bbox = []

# Membaca dan memproses file JSON baris per baris
with open(metrics_file, 'r') as f:
    for line in f:
        entry = json.loads(line)

        if 'iteration' in entry:
            iterations.append(entry['iteration'])
            total_loss.append(entry.get('total_loss', None))
            loss_cls.append(entry.get('loss_cls', None))
            loss_bbox.append(entry.get('loss_box_reg', None))

# Membuat grafik
plt.figure(figsize=(12, 7))

plt.plot(iterations, total_loss, label='Total Loss', color='tab:blue')
plt.plot(iterations, loss_cls, label='Classification Loss (loss_cls)', color='tab:orange')
plt.plot(iterations, loss_bbox, label='BBox Loss (loss_bbox)', color='tab:green')

# Menambahkan elemen visual
plt.title('Loss over Iterations')
plt.xlabel('Iteration')
plt.ylabel('Loss Value')
plt.legend()
plt.grid(True)
plt.tight_layout()

# Menampilkan grafik
plt.show()


In [ ]:
import json
import matplotlib.pyplot as plt

# Tentukan path ke file metrics.json
metrics_file = '/content/drive/MyDrive/210411100014_Taufiqu Reza Yoga Pratama/Model/new/epoch_30_1e-5_Adam/metrics.json'  # Ganti dengan path file Anda

# Menyaring data untuk AP
iterations = []
bbox_ap = []
bbox_ap50 = []
bbox_ap75 = []
bbox_ap_cardio = []
bbox_ap_nodule = []
bbox_ap_pneumo = []

# Membaca file JSON baris per baris
with open(metrics_file, 'r') as f:
    for line in f:
        try:
            entry = json.loads(line)

            # Pastikan hanya memasukkan data yang mengandung metrik 'bbox/AP'
            if 'bbox/AP' in entry:
                iterations.append(entry['iteration'])
                bbox_ap.append(entry['bbox/AP'])
                bbox_ap50.append(entry.get('bbox/AP50', None))
                bbox_ap75.append(entry.get('bbox/AP75', None))
                bbox_ap_cardio.append(entry.get('bbox/AP-Cardiomegaly', None))
                bbox_ap_nodule.append(entry.get('bbox/AP-Nodule-Mass', None))
                bbox_ap_pneumo.append(entry.get('bbox/AP-Pneumothorax', None))

        except json.JSONDecodeError as e:
            print(f"Error parsing line: {e}")
            continue

# Memastikan ada data untuk plotting
print(f"Total data points: {len(iterations)}")

# Jika ada data untuk plotting, buat grafik
if iterations:
    plt.figure(figsize=(12, 8))

    # Plotting berbagai metrik AP
    plt.plot(iterations, bbox_ap, label="bbox/AP", color='tab:blue')
    plt.plot(iterations, bbox_ap50, label="bbox/AP50", color='tab:orange')
    plt.plot(iterations, bbox_ap75, label="bbox/AP75", color='tab:green')
    plt.plot(iterations, bbox_ap_cardio, label="bbox/AP-Cardiomegaly", color='tab:red')
    plt.plot(iterations, bbox_ap_nodule, label="bbox/AP-Nodule-Mass", color='tab:purple')
    plt.plot(iterations, bbox_ap_pneumo, label="bbox/AP-Pneumothorax", color='tab:brown')

    # Menambahkan judul dan label sumbu
    plt.title('Average Precision (AP) over Iterations')
    plt.xlabel('Iteration')
    plt.ylabel('Average Precision (AP)')
    plt.grid(True)

    # Menampilkan legenda
    plt.legend()

    # Menampilkan grafik
    plt.show()
else:
    print("Tidak ada data untuk plot.")


In [ ]:
import json

# Menentukan path file metrics.json di Google Drive
file_path = '/content/drive/MyDrive/210411100014_Taufiqu Reza Yoga Pratama/Model/new/epoch_30_1e-5_Adam/metrics.json'

# Memuat file metrics.json
with open(file_path, 'r') as file:
    data = [json.loads(line) for line in file.readlines()]

# Menyaring nilai AP per kelas dari data terakhir
metrics = data[-1]  # Mengambil data terakhir yang berisi AP

# Mengambil nilai AP per kelas
ap_values = {
    "Cardiomegaly": metrics.get('bbox/AP-Cardiomegaly', 0),
    "Nodule-Mass": metrics.get('bbox/AP-Nodule-Mass', 0),
    "Pneumothorax": metrics.get('bbox/AP-Pneumothorax', 0),
}

# Menghitung mAP (mean Average Precision)
ap_values_list = list(ap_values.values())
map_score = sum(ap_values_list) / len(ap_values_list)

# Menampilkan hasil
print("AP per kelas:")
for class_name, ap in ap_values.items():
    print(f"{class_name}: {ap:.4f}")

print(f"\nmAP: {map_score:.4f}")


In [ ]:
import json
import matplotlib.pyplot as plt

# Menentukan path file metrics.json (update dengan path yang sesuai)
file_path = '/content/drive/MyDrive/210411100014_Taufiqu Reza Yoga Pratama/Model/new/epoch_30_1e-5_Adam/metrics.json'

# Memuat file metrics.json
with open(file_path, 'r') as file:
    data = [json.loads(line) for line in file.readlines()]

# Menyaring nilai AP per kelas dari data terakhir
ap_per_iteration = []
map_per_iteration = []

# Menyaring nilai AP per kelas dan mAP dari setiap iterasi
for metrics in data:
    # Mengecek apakah nilai AP ada dalam data
    if 'bbox/AP-Cardiomegaly' in metrics and 'bbox/AP-Nodule-Mass' in metrics and 'bbox/AP-Pneumothorax' in metrics:
        ap_values = {
            "Cardiomegaly": metrics.get('bbox/AP-Cardiomegaly', 0),
            "Nodule-Mass": metrics.get('bbox/AP-Nodule-Mass', 0),
            "Pneumothorax": metrics.get('bbox/AP-Pneumothorax', 0),
        }

        # Menghitung mAP (mean Average Precision)
        ap_values_list = list(ap_values.values())
        map_score = sum(ap_values_list) / len(ap_values_list)

        ap_per_iteration.append(ap_values)
        map_per_iteration.append(map_score)

# Menyiapkan data untuk grafik
iterations = range(1, len(map_per_iteration) + 1)

# Plotting mAP per iterasi
plt.figure(figsize=(10, 6))
plt.plot(iterations, map_per_iteration, label='mAP (mean Average Precision)', color='blue', linewidth=2)

# Menambahkan grafik untuk setiap kelas
for class_name in ap_per_iteration[0].keys():
    class_ap_values = [ap[class_name] for ap in ap_per_iteration]
    plt.plot(iterations, class_ap_values, label=f'AP-{class_name}', linestyle='--')

# Menambahkan label dan judul
plt.xlabel('Iterations')
plt.ylabel('Average Precision')
plt.title('mAP and AP per Class over Iterations')
plt.legend(loc='upper right')

# Menampilkan grafik
plt.grid(True)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import json

# Path file di Google Drive (sesuaikan dengan path file Anda)
file_path = '/content/drive/MyDrive/210411100014_Taufiqu Reza Yoga Pratama/Model/new/epoch_30_1e-5_Adam/metrics.json'

# Memuat data dari file metrics.json
with open(file_path, 'r') as f:
    data = [json.loads(line) for line in f]

# Menyiapkan data untuk plot
iterations = []
loss_cls_values = []
loss_box_reg_values = []

# Looping untuk mengekstrak data
for metrics in data:
    # Pastikan ada key yang dibutuhkan
    if 'iteration' in metrics:
        iterations.append(metrics['iteration'])
    if 'loss_cls' in metrics:
        loss_cls_values.append(metrics['loss_cls'])
    if 'loss_box_reg' in metrics:
        loss_box_reg_values.append(metrics['loss_box_reg'])

# Pastikan panjang data sama, dan jika tidak, sesuaikan panjangnya
min_len = min(len(iterations), len(loss_cls_values), len(loss_box_reg_values))

# Potong data agar memiliki panjang yang sama
iterations = iterations[:min_len]
loss_cls_values = loss_cls_values[:min_len]
loss_box_reg_values = loss_box_reg_values[:min_len]

# Membuat plot jika data ada
if iterations and loss_cls_values and loss_box_reg_values:
    plt.figure(figsize=(10, 6))

    # Plot Loss Klasifikasi
    plt.plot(iterations, loss_cls_values, label='Loss Klasifikasi', color='red', linestyle='--', linewidth=2)

    # Plot Loss Regresi Batas
    plt.plot(iterations, loss_box_reg_values, label='Loss Regresi Bounding Box', color='green', linestyle='-', linewidth=2)

    # Menambahkan label dan judul
    plt.xlabel('Iterations')
    plt.ylabel('Loss')
    plt.title('Loss Klasifikasi dan Loss Regresi Bounding Box')

    # Menambahkan legend
    plt.legend(loc='upper right')

    # Menampilkan grid
    plt.grid(True)

    # Menampilkan grafik
    plt.show()
else:
    print("Data tidak lengkap, pastikan key 'loss_cls' dan 'loss_box_reg' ada dalam setiap entri.")


In [ ]:
import json
import matplotlib.pyplot as plt

# Path file metrics.json
file_path = '/content/drive/MyDrive/210411100014_Taufiqu Reza Yoga Pratama/Model/new/epoch_30_1e-5_Adam/metrics.json'

# Load file JSON
with open(file_path, 'r') as file:
    data = [json.loads(line) for line in file.readlines()]

# Menyimpan AP per kelas dan iterasi
ap_per_iteration = { "Cardiomegaly": [], "Nodule-Mass": [], "Pneumothorax": [] }
iteration_list = []

# Ekstraksi AP dan iterasi
for metrics in data:
    if all(k in metrics for k in ['bbox/AP-Cardiomegaly', 'bbox/AP-Nodule-Mass', 'bbox/AP-Pneumothorax']):
        iteration = metrics.get("iteration", None)
        if iteration is not None:
            iteration_list.append(iteration)
            ap_per_iteration["Cardiomegaly"].append(metrics['bbox/AP-Cardiomegaly'])
            ap_per_iteration["Nodule-Mass"].append(metrics['bbox/AP-Nodule-Mass'])
            ap_per_iteration["Pneumothorax"].append(metrics['bbox/AP-Pneumothorax'])

# Plot untuk setiap kelas
for class_name, ap_values in ap_per_iteration.items():
    plt.figure(figsize=(8, 5))
    plt.plot(iteration_list, ap_values, marker='o', color='red', label=f'bbox/AP-{class_name}')
    plt.xlabel('Iteration')
    plt.ylabel('Average Precision (AP)')
    plt.title(f'Average Precision (AP) {class_name} over Iterations')
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()


# **Confusion Matrix**

In [ ]:
def calculate_iou(box1, box2):
    """
    box: [xmin, ymin, xmax, ymax]
    """
    x1, y1, x2, y2 = box1
    x1g, y1g, x2g, y2g = box2

    xi1 = max(x1, x1g)
    yi1 = max(y1, y1g)
    xi2 = min(x2, x2g)
    yi2 = min(y2, y2g)
    inter_area = max(xi2 - xi1, 0) * max(yi2 - yi1, 0)

    box1_area = (x2 - x1) * (y2 - y1)
    box2_area = (x2g - x1g) * (y2g - y1g)
    union_area = box1_area + box2_area - inter_area

    if union_area == 0:
        return 0
    else:
        return inter_area / union_area


In [ ]:
import os
import cv2
import numpy as np
import json
import matplotlib.pyplot as plt
from tqdm import tqdm
from detectron2.engine import DefaultPredictor
from detectron2.data import MetadataCatalog
from detectron2.utils.visualizer import Visualizer
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# 🧠 Load model
# cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "best_model.pth")
cfg.MODEL.WEIGHTS = "/content/drive/MyDrive/210411100014_Taufiqu Reza Yoga Pratama/Model/new/epoch_30_5e-5_Adam/best_model.pth"
predictor = DefaultPredictor(cfg)

# 📚 Prepare dataset paths
test_image_dir = "/content/drive/MyDrive/210411100014_Taufiqu Reza Yoga Pratama/Dataset/dataset_new/test"
coco_annotation_path = "/content/drive/MyDrive/210411100014_Taufiqu Reza Yoga Pratama/Dataset/dataset_new/test/_annotations.coco.json"

# Load COCO ground truth
with open(coco_annotation_path) as f:
    coco_data = json.load(f)

# Build image_id to filename mapping
id_to_filename = {img['id']: img['file_name'] for img in coco_data['images']}

# Build annotations mapping
annotations = coco_data['annotations']
gt_dict = {}  # {filename: (boxes, labels)}

for ann in annotations:
    image_id = ann['image_id']
    filename = id_to_filename[image_id]
    bbox = ann['bbox']  # [x, y, width, height]
    category_id = ann['category_id']

    if filename not in gt_dict:
        gt_dict[filename] = {'boxes': [], 'labels': []}

    # Convert bbox format to [xmin, ymin, xmax, ymax]
    xmin, ymin, w, h = bbox
    xmax = xmin + w
    ymax = ymin + h

    gt_dict[filename]['boxes'].append([xmin, ymin, xmax, ymax])
    gt_dict[filename]['labels'].append(category_id)

# Build category_id to class_name mapping
category_id_to_name = {cat['id']: cat['name'] for cat in coco_data['categories']}
class_names = [category_id_to_name[i] for i in sorted(category_id_to_name.keys())]

# 🔥 Start evaluating
y_true = []
y_pred = []
iou_threshold = 0.5
confidence_threshold = 0.5

test_images = sorted(os.listdir(test_image_dir))
test_images = [img for img in test_images if img.endswith('.jpg') or img.endswith('.png')]

for img_file in tqdm(test_images, desc="Processing Test Images"):
    # Read image
    img_path = os.path.join(test_image_dir, img_file)
    img = cv2.imread(img_path)

    if img_file not in gt_dict:
        continue  # skip images without ground truth

    gt_boxes = np.array(gt_dict[img_file]['boxes'])
    gt_classes = np.array(gt_dict[img_file]['labels'])

    # Predict
    outputs = predictor(img)
    instances = outputs["instances"].to("cpu")

    pred_boxes = instances.pred_boxes.tensor.numpy()
    pred_classes = instances.pred_classes.numpy()
    pred_scores = instances.scores.numpy()

    # Filter predictions by confidence
    keep = pred_scores > confidence_threshold
    pred_boxes = pred_boxes[keep]
    pred_classes = pred_classes[keep]

    matched_gt = set()

    # Matching predicted boxes to ground truth boxes
    for pred_box, pred_class in zip(pred_boxes, pred_classes):
        best_iou = 0
        best_idx = -1
        for idx, gt_box in enumerate(gt_boxes):
            iou = calculate_iou(pred_box, gt_box)
            if iou > best_iou and idx not in matched_gt:
                best_iou = iou
                best_idx = idx
        if best_iou >= iou_threshold and best_idx != -1:
            y_true.append(gt_classes[best_idx])
            y_pred.append(pred_class)
            matched_gt.add(best_idx)
        else:
            # False positive
            y_true.append(-1)  # background
            y_pred.append(pred_class)

    # False negatives
    for idx, gt_class in enumerate(gt_classes):
        if idx not in matched_gt:
            y_true.append(gt_class)
            y_pred.append(-1)

# 🔥 Build Confusion Matrix
all_labels = sorted(list(set([lab for lab in y_true if lab != -1] + [lab for lab in y_pred if lab != -1])))
cm = confusion_matrix(y_true, y_pred, labels=all_labels)

class_labels = [category_id_to_name[i] for i in all_labels]

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_labels)
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix - RetinaNet Detectron2 (COCO Format)")
plt.show()


# **Evaluasi Data Train, Validasi, dan Testing**

In [ ]:
from detectron2.config import get_cfg
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.modeling import build_model

# Load config dan model
cfg = get_cfg()
cfg.merge_from_file("/content/drive/MyDrive/210411100014_Taufiqu Reza Yoga Pratama/Model/new/epoch_30_5e-5_Adam/config.yaml")
cfg.MODEL.WEIGHTS = "/content/drive/MyDrive/210411100014_Taufiqu Reza Yoga Pratama/Model/new/epoch_30_5e-5_Adam/best_model.pth"
cfg.MODEL.DEVICE = 'cpu'  # atau 'cpu'
cfg.DATASETS.TEST = ("roboflow_train",)
cfg.MODEL.RETINANET.SCORE_THRESH_TEST = 0.5

evaluator = COCOEvaluator("roboflow_train", cfg, False, output_dir="/content/drive/MyDrive/210411100014_Taufiqu Reza Yoga Pratama/Model/new/epoch_30_5e-5_Adam/eval_train")
val_loader = build_detection_test_loader(cfg, "roboflow_train")

model = build_model(cfg)
DetectionCheckpointer(model).load(cfg.MODEL.WEIGHTS)

results = inference_on_dataset(model, val_loader, evaluator)
print(results)

In [ ]:
from detectron2.config import get_cfg
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.modeling import build_model

# Load config dan model
cfg = get_cfg()
cfg.merge_from_file("/content/drive/MyDrive/210411100014_Taufiqu Reza Yoga Pratama/Model/new/epoch_30_5e-5_Adam/config.yaml")
cfg.MODEL.WEIGHTS = "/content/drive/MyDrive/210411100014_Taufiqu Reza Yoga Pratama/Model/new/epoch_30_5e-5_Adam/best_model.pth"
cfg.MODEL.DEVICE = 'cpu'  # atau 'cpu'
cfg.DATASETS.TEST = ("roboflow_val",)
cfg.MODEL.RETINANET.SCORE_THRESH_TEST = 0.5

evaluator = COCOEvaluator("roboflow_val", cfg, False, output_dir="/content/drive/MyDrive/210411100014_Taufiqu Reza Yoga Pratama/Model/new/epoch_30_5e-5_Adam/eval_valid")
val_loader = build_detection_test_loader(cfg, "roboflow_val")

model = build_model(cfg)
DetectionCheckpointer(model).load(cfg.MODEL.WEIGHTS)

results = inference_on_dataset(model, val_loader, evaluator)
print(results)

In [ ]:
from detectron2.config import get_cfg
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.modeling import build_model

# Load config dan model
cfg = get_cfg()
cfg.merge_from_file("/content/drive/MyDrive/210411100014_Taufiqu Reza Yoga Pratama/Model/new/epoch_30_5e-5_Adam/config.yaml")
cfg.MODEL.WEIGHTS = "/content/drive/MyDrive/210411100014_Taufiqu Reza Yoga Pratama/Model/new/epoch_30_5e-5_Adam/best_model.pth"
cfg.MODEL.DEVICE = 'cpu'  # atau 'cpu'
cfg.DATASETS.TEST = ("roboflow_test",)
cfg.MODEL.RETINANET.SCORE_THRESH_TEST = 0.5

evaluator = COCOEvaluator("roboflow_test", cfg, False, output_dir="/content/drive/MyDrive/210411100014_Taufiqu Reza Yoga Pratama/Model/new/epoch_30_5e-5_Adam/eval_test")
val_loader = build_detection_test_loader(cfg, "roboflow_test")

model = build_model(cfg)
DetectionCheckpointer(model).load(cfg.MODEL.WEIGHTS)

results = inference_on_dataset(model, val_loader, evaluator)
print(results)

In [ ]:
from detectron2.evaluation.evaluator import DatasetEvaluator
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd

class PRF1Evaluator(DatasetEvaluator):
    def __init__(self, dataset_name, metadata, class_names=None):
        from detectron2.data import MetadataCatalog, DatasetCatalog
        self.metadata = MetadataCatalog.get(dataset_name)
        self.dataset_dicts = DatasetCatalog.get(dataset_name)
        self.class_names = class_names if class_names else self.metadata.thing_classes
        self.gts = []
        self.preds = []

        # Buat mapping image_id ke daftar ground truth class ID
        self.imgid_to_gt = {}
        for data in self.dataset_dicts:
            gt_classes = [ann["category_id"] for ann in data["annotations"]]
            self.imgid_to_gt[data["image_id"]] = gt_classes

    def reset(self):
        self.gts = []
        self.preds = []

    def process(self, inputs, outputs):
        for input, output in zip(inputs, outputs):
            img_id = input["image_id"]
            gt_classes = self.imgid_to_gt.get(img_id, [])
            pred_classes = output["instances"].pred_classes.cpu().numpy() if len(output["instances"]) > 0 else []

            # Samakan panjang prediksi dan ground truth
            if len(pred_classes) < len(gt_classes):
                pred_classes = list(pred_classes) + [-1] * (len(gt_classes) - len(pred_classes))
            elif len(pred_classes) > len(gt_classes):
                pred_classes = pred_classes[:len(gt_classes)]

            self.gts.extend(gt_classes)
            self.preds.extend(pred_classes)

    def evaluate(self):
        print("\n==== Precision, Recall, and F1-Score per Class ====\n")

        # Ambil hasil sebagai dictionary
        report_dict = classification_report(
            self.gts,
            self.preds,
            target_names=self.class_names,
            output_dict=True,
            digits=3,
            zero_division=0
        )

        # Konversi ke DataFrame
        df = pd.DataFrame(report_dict).transpose()
        df.index.name = 'Class'

        # Ambil dan simpan nilai akurasi, hapus dari tabel supaya tampil rapi
        accuracy = report_dict.get("accuracy", None)
        if "accuracy" in df.index:
            df.drop("accuracy", inplace=True)

        # Ubah support menjadi integer
        if 'support' in df.columns:
            df['support'] = df['support'].astype(int)

        # Urutkan kolom
        df = df[['precision', 'recall', 'f1-score', 'support']]

        # Cetak tabel
        print(df.to_string(float_format="%.3f"))

        # Cetak akurasi secara terpisah
        if accuracy is not None:
            print(f"\nOverall Accuracy: {accuracy:.3f}")

        # Return hanya data, tanpa tampilkan dict sebagai output Python
        return


In [ ]:
class_names = ["Cardiomegaly", "Nodule/Mass", "Pneumothorax"]

evaluator_test = PRF1Evaluator("roboflow_train", cfg, class_names=class_names)
test_loader = build_detection_test_loader(cfg, "roboflow_train")
inference_on_dataset(trainer.model, test_loader, evaluator_test)


In [ ]:
class_names = ["Cardiomegaly", "Nodule/Mass", "Pneumothorax"]

evaluator_test = PRF1Evaluator("roboflow_val", cfg, class_names=class_names)
test_loader = build_detection_test_loader(cfg, "roboflow_val")
inference_on_dataset(trainer.model, test_loader, evaluator_test)

In [ ]:
class_names = ["Cardiomegaly", "Nodule/Mass", "Pneumothorax"]

evaluator_test = PRF1Evaluator("roboflow_test", cfg, class_names=class_names)
test_loader = build_detection_test_loader(cfg, "roboflow_test")
inference_on_dataset(trainer.model, test_loader, evaluator_test)